*** 머신러닝 시작하기 ***

○ 머신러닝: 경험을 통해 자동으로 개선하는 컴퓨터 알고리즘의 연구. 인공지능의 한 분야이며, 컴퓨터가 학습할 수 있도록 하는 알고리즘과 기술을 개발하는 분야.

○ 목적: 그레이스케일로 변환된 데이터를 머신러닝으로 시행해서 적합성과 각종 문제점을 알아보기 위함.

○ 데이터 셋 설명
- 손상 컨테이너: 277
- 정상 컨테이너: 299
- 데이터 크기 : 128 * 128 * 1 (1차원)

In [ ]:
import numpy as np
import pandas as pd
X = np.load('ml_dnn_con_X.npy')
Y = np.load('ml_dnn_con_Y.npy')

In [ ]:
print('input의 행렬: ', X.shape)
print('target의 행렬: ', Y.shape)
print('target의 클래스:', np.unique(Y))

input의 행렬:  (576, 65536)
target의 행렬:  (576, 1)
target의 클래스: [0. 1.]


In [ ]:
# 타겟은 벡터로 바꾸어야, 머신러닝이 가능한 것 같음
Y = Y.flatten()
print("target의 벡터: ", Y.shape)

target의 벡터:  (576,)


## 1. 머신러닝 시작하기

- con_X와 con_Y 파일은 사진데이터의 픽셀데이터로 구성된 값.
- 훈련 세트, 테스트 세트 분류 및 표준화 진행.

In [ ]:
# 훈련세트와 테스트세트 분류
# 분류 조건은 훈련 세트(90%), 테스트 세트(10%), 분할 비율 동일하게 유지
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size = 0.1, random_state=42)

In [ ]:
# 데이터 표준화(0~255)
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

### 1.1. K-최근접 이웃분류(게으른 학습 알고리즘)
- K-최근접 이웃 분류는 입력된 값이 위치한 곳에서 가장 가까운 데이터 k개를 고른 뒤, 과반수 의결을 통해 출력값을 정하는 알고리즘.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors= 100, metric="euclidean")
#정규화 된 데이터셋 사용
knn.fit(X_train_scaled, Y_train)

print("훈련 세트의 정 확 도: ", knn.score(X_train_scaled, Y_train))
print("테스트 세트의 정확도: ", knn.score(X_test_scaled, Y_test))

훈련 세트의 정 확 도:  0.7123552123552124
테스트 세트의 정확도:  0.7068965517241379


### 1.2 로지스틱 회귀분석
- 여러 특성을 고려했을 때 특정값 A로 분류될 확률을 구하는 알고리즘. 이때 여러 특성이 고려된 관계식은 선형방정식을 사용.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', multi_class='auto', C=0.00000001, random_state= 42)

In [ ]:
#정규화 된 데이터셋 사용
lr.fit(X_train_scaled, Y_train)

LogisticRegression(C=1e-08, random_state=42, solver='liblinear')

In [ ]:
#결과 예측
lr.predict_proba(X_test[:5, :])

array([[0.2250811 , 0.7749189 ],
       [0.23041082, 0.76958918],
       [0.18174892, 0.81825108],
       [0.22883234, 0.77116766],
       [0.19920788, 0.80079212]])

In [ ]:
print("훈련 세트의 정 확 도: ", lr.score(X_train_scaled, Y_train))
print("테스트 세트의 정확도: ", lr.score(X_test_scaled, Y_test))

훈련 세트의 정 확 도:  0.5193050193050193
테스트 세트의 정확도:  0.5172413793103449


In [ ]:
print("로지스틱 회귀계수 값 (절편:", lr.intercept_, "), \n (계수: ", lr.coef_, ")")

로지스틱 회귀계수 값 (절편: [9.32595823e-08] ), 
 (계수:  [[2.63727448e-07 2.68602332e-07 2.73208057e-07 ... 3.17831425e-07
  3.14767705e-07 3.01994202e-07]] )


### 1.3 앙상블(랜덤 포레스트)
- 결정트리를 랜덤하게 만들어 결정트리의 숲을 만들고 각 결정트리의 예측을 사용해 최종 예측을 만드는 알고리즘. 결정트리의 훈련데이터에 과대적합 되는 경향을 줄임.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
#정규화 되지 않은 데이터셋 사용
scores=cross_validate(rf, X_train, Y_train, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

1.0 0.756628080657207


In [ ]:
rf.fit(X_train, Y_train)
#중요도 출력
print(rf.feature_importances_)

[0.00000000e+00 9.19270800e-05 0.00000000e+00 ... 0.00000000e+00
 1.41763663e-04 0.00000000e+00]


In [ ]:
#OOB 점수 출력
rf=RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(X_train, Y_train)
print(rf.oob_score_)

0.7432432432432432


### 1.4 앙상블(XGBoost)
- 결정트리를 기반으로 한 앙상블, 그중에서도 부스팅을 기반으로 하는 머신러닝 알고리즘. 부스팅은 약한 분류기를 세트로 묶어 정확도를 예측하는 기법.

In [ ]:
#!pip install xgboost

In [ ]:
#정규화 되지 않은 데이터셋 사용
from xgboost import XGBClassifier

xgb=XGBClassifier(n_estimators=200,max_depth=3,eval_metric='mlogloss')
xgb.fit(X_train, Y_train)


In [ ]:
scores=cross_validate(xgb, X_train, Y_train, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

### 1.5 하이퍼 파라미터튜닝(그리드 서치)
○ 하이퍼파라미터 튜닝: 사람이 지정해야 할 튜닝 옵션을 하이퍼파라미터라고 부르며 성능을 최적화하거나 편향과 분산 사이의 균형을 맞출 때 알고리즘을 조절하는 것
- 그리드 서치: 가능한 모든 하이퍼파라미터 조합에 대해 교차 검증을 사용해 평가함.


In [ ]:
from sklearn.model_selection import GridSearchCV
params1={'n_neighbors':range(50,150,2)}   #K-최근접이웃 분류
params2={'C':[0.00001,0.0001,0.001,0.01]}   #로지스틱 회귀
params3={'max_depth' : range(6,13,1) , 'n_estimators': range(60,110,10)} #랜덤 포레스트
params4={'max_depth' : range(1,10,1) , 'gamma' : [0.01,0.1,0,1,10] ,        
         'n_estimators' : range(100,1000,100) , 'min_child_weight' : range(1,4,1)}     #XGBoost

In [ ]:
gs1=GridSearchCV(KNeighborsClassifier(metric="minkowski"),params1,n_jobs=-1)
gs1.fit(X_train_scaled, Y_train)

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(50, 150, 2)})

In [ ]:
#제일 결과가 좋은 파라미터
print(gs1.best_params_)

{'n_neighbors': 54}


In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt1=gs1.best_estimator_
print(dt1.score(X_train_scaled, Y_train))
print(dt1.score(X_test_scaled, Y_test))

0.7239382239382239
0.6551724137931034


In [ ]:
gs2=GridSearchCV(LogisticRegression(solver='liblinear', multi_class='auto',random_state=42),params2,n_jobs=-1)
gs2.fit(X_train_scaled, Y_train)

GridSearchCV(estimator=LogisticRegression(random_state=42, solver='liblinear'),
             n_jobs=-1, param_grid={'C': [1e-05, 0.0001, 0.001, 0.01]})

In [ ]:
#제일 결과가 좋은 파라미터
print(gs2.best_params_)

{'C': 0.001}


In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt2=gs2.best_estimator_
print(dt2.score(X_train_scaled, Y_train))
print(dt2.score(X_test_scaled, Y_test))

0.8687258687258688
0.5862068965517241


In [ ]:
gs3=GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=42),params3,n_jobs=-1)
gs3.fit(X_train, Y_train)

In [ ]:
#제일 결과가 좋은 파라미터
print(gs3.best_params_)

In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt3=gs3.best_estimator_
print(dt3.score(X_train, Y_train))
print(dt3.score(X_test, Y_test))

In [ ]:
gs4=GridSearchCV(XGBClassifier(random_state=42),params4,n_jobs=-1)
gs4.fit(X_train, Y_train)

C:\Users\152c1\anaconda3\envs\tf24\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:29:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=42,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameters=

In [ ]:
#제일 결과가 좋은 파라미터
print(gs4.best_params_)

{'gamma': 0.01, 'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 1000}


In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt4=gs4.best_estimator_
print(dt4.score(X_train, Y_train))
print(dt4.score(X_test, Y_test))

1.0
0.6724137931034483


### 1.6 SVM(Support Vector Machine)
- 결정 경계(Decision Boundary), 즉 분류를 위한 기준 선을 정의하는 모델. 분류되지 않은 새로운 점이 나타나면 경계의 어느 쪽에 속하는지 확인해서 분류 과제를 수행.

In [ ]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate

# SVM, kernel = 'linear'로 선형분리 진행
 
svm_clf =svm.SVC(kernel = 'linear')

# 교차검증

scores = cross_val_score(svm_clf, X_train_scaled, Y_train, cv = 5)
scores

pd.DataFrame(cross_validate(svm_clf, X_train_scaled, Y_train, cv =5))

print('교차검증 평균: ', scores.mean())

교차검증 평균:  0.685418222554145


In [ ]:
svm_clf.fit(X_train_scaled, Y_train)

print('학습용 데이터로 측정한 정확도 = %.2f' % svm_clf.score(X_train_scaled, Y_train))
print('시험용 데이터로 측정한 정확도 = %.2f' % svm_clf.score(X_test_scaled, Y_test))

학습용 데이터로 측정한 정확도 = 1.00
시험용 데이터로 측정한 정확도 = 0.64


### 하이퍼파라미터 튜닝 랜덤서치
- 랜덤서치: 하이퍼 파라미터 값을 랜덤하게 넣어보고 그중 우수한 값을 보인 하이퍼 파라미터를 활용해 모델을 생성.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
params1={'n_neighbors':range(10,100,2)}   #K-최근접 이웃 분류
params2={'C':[0.00001,0.0001,0.001,0.01]} #로지스틱 회귀
params3={'max_depth' : range(3,7,1) , 'n_estimators': range(30,110,10)}  #랜덤 포레스트
params4={'max_depth' : range(1,10,1) , 'gamma' : [0.01,0.1,0,1,10] , 
         'n_estimators' : range(100,1000,100) , 'min_child_weight' : range(1,4,1)}  #XGBoost

In [ ]:
rs1=RandomizedSearchCV(KNeighborsClassifier(metric="minkowski"),params1,n_jobs=-1)
rs1.fit(X_train_scaled, Y_train)

RandomizedSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'n_neighbors': range(10, 100, 2)})

In [ ]:
#제일 결과가 좋은 파라미터
print(rs1.best_params_)  

{'n_neighbors': 56}


In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt1=rs1.best_estimator_
print(dt1.score(X_train_scaled, Y_train))
print(dt1.score(X_test_scaled, Y_test))

0.7162162162162162
0.6379310344827587


In [ ]:
rs2=RandomizedSearchCV(LogisticRegression(solver='liblinear', multi_class='auto',random_state=42),params2,n_jobs=-1)
rs2.fit(X_train_scaled, Y_train)

D:\Anaconda\envs\tf24\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=LogisticRegression(random_state=42,
                                                solver='liblinear'),
                   n_jobs=-1,
                   param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01]})

In [ ]:
#제일 결과가 좋은 파라미터
print(rs2.best_params_)

{'C': 0.001}


In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt2=rs2.best_estimator_
print(dt2.score(X_train_scaled, Y_train))
print(dt2.score(X_test_scaled, Y_test))

0.8687258687258688
0.5862068965517241


In [ ]:
rs3=RandomizedSearchCV(RandomForestClassifier(n_jobs=-1, random_state=42),params3,n_jobs=-1)
rs3.fit(X_train, Y_train)

In [ ]:
#제일 결과가 좋은 파라미터
print(rs3.best_params_)

In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt3=rs3.best_estimator_
print(dt3.score(X_train, Y_train))
print(dt3.score(X_test, Y_test))

In [ ]:
rs4=RandomizedSearchCV(XGBClassifier(random_state=42),params4,n_jobs=-1)
rs4.fit(X_train, Y_train)

In [ ]:
#제일 결과가 좋은 파라미터
print(rs4.best_params_)

In [ ]:
#제일 결과가 좋은 파라미터를 넣어본 결과
dt4=rs4.best_estimator_
print(dt4.score(X_train, Y_train))
print(dt4.score(X_test, Y_test))

### 1.6 sonar 데이터를 활용한 비교
○ 컨테이너 이미지 데이터와 유사하게 분류 데이터인 sonar 데이터로 비교

In [ ]:
import pandas as pd
sonar = pd.read_csv('./dataset/sonar.csv', header=None)
data=sonar.values

In [ ]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy
input=data[:,:60]
target=data[:, 60]
e = LabelEncoder()
e.fit(target)
target = e.transform(target)

In [ ]:
# 훈련세트와 테스트세트 분류
# 분류 조건은 훈련 세트(90%), 테스트 세트(10%), 분할 비율 동일하게 유지
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(input, target, stratify=target, test_size = 0.1, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train_input)
train_std = sc.transform(train_input)
target_std = sc.transform(test_input)

- K-최근접 이웃 분류

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_std, train_target)

print("훈련 세트의 정 확 도: ", knn.score(train_std, train_target))
print("테스트 세트의 정확도: ", knn.score(target_std, test_target))

훈련 세트의 정 확 도:  0.9197860962566845
테스트 세트의 정확도:  0.8095238095238095


- 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', multi_class='auto', random_state= 42)

In [ ]:
lr.fit(train_std, train_target)

print("훈련 세트의 정 확 도: ", lr.score(train_std, train_target))
print("테스트 세트의 정확도: ", lr.score(target_std, test_target))

훈련 세트의 정 확 도:  0.9090909090909091
테스트 세트의 정확도:  0.8571428571428571


- 랜덤 포레스트

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)

scores=cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

1.0 0.8018492176386914


- XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb=XGBClassifier()
xgb.fit(train_input, train_target)

scores=cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

[15:15:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Anaconda\envs\tf24\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1.0 0.8073968705547653


- SVM

In [ ]:
import sklearn.svm as svm
import sklearn.metrics as mt
from sklearn.model_selection import cross_val_score, cross_validate

# SVM, kernel = 'linear'로 선형분리 진행
 
svm_clf =svm.SVC(kernel = 'linear')

# 교차검증

scores = cross_val_score(svm_clf, X_train_scaled, Y_train, cv = 5)
scores

pd.DataFrame(cross_validate(svm_clf, X_train_scaled, Y_train, cv =5))

print('교차검증 평균: ', scores.mean())